# DNN 모델을 이용하여 동영상 파일 내의 얼굴에 사각형 및 원 표시를 추가 및 플레이

In [1]:
# 필요모듈 import
import cv2
import numpy as np
model_name = 'res10_300x300_ssd_iter_140000.caffemodel'
prototxt_name = 'deploy.prototxt.txt'

In [9]:
# 얼굴인식 모델 경로 및 min_confidense 설정(동영상 파일 경로 및 파일명 저장)
min_confidence = 0.8
file_name = "video/obama_01.mp4"

In [10]:
face_cascade = cv2.CascadeClassifier()
face_cascade_name = './data/haarcascade_frontalface_alt.xml'
# 모듈로딩 : 실패하면 프로그램 종료
if not face_cascade.load(cv2.samples.findFile(face_cascade_name)):
    print('--(!)Error loading face cascade')
    exit(0)

In [11]:
# detectAndDisplay 함수 제작(이미지 형식 변환 및 얼굴인식 표시 추가  & display)
def detectAndDisplay(frame):
    model = cv2.dnn.readNetFromCaffe(prototxt_name, model_name)
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (500, 300)), 1.0, (500, 300), \
                                (104.0, 177.0, 123.0))
    model.setInput(blob)
    detections = model.forward()  # 얼굴인식에 의한 리턴값 리스트
    # print(detections.shape)
    # print(detections[0])

    for i in range(0, detections.shape[2]):
        confidence = detections[0,0,i,2]
        if confidence > min_confidence:
            box = detections[0,0,i,3:7] * np.array([width, height, width, height])
            (startX, startY, endX, endY) = box.astype("int")
            text = "{:.2f}%".format(confidence*100)
            # 박스가 너무 위에 있어 확률표시가 어렵다면 박스 안쪽에 확률표시
            y = startY - 10 if startY- 10 > 10 else startY + 10
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0,255,0), 2)
            cv2.putText(frame, text, (startX, y), 
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    cv2.imshow('Capture - Face detection by DNN', frame)

In [ ]:
# 동영상 캡쳐화면을 이용한 detectAndDisplay 함수 호출
cap = cv2.VideoCapture(file_name)
if not cap.isOpened:
    print('--(!)Error opening video cascade')
    exit(0)

while True:
    ret, frame = cap.read()
    if frame is None:
        print('--(!) No captured frame -- Break!')
        break
        
    (height, width) = frame.shape[:2]
    detectAndDisplay(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()